In [35]:
import pandas as pd
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import recordlinkage

lemma = WordNetLemmatizer()

In [36]:
def text_preprocess(text_):
    sent=[]
    name=[]
    count=0
    for text in text_:
        stop_words = stopwords.words("english")+list(punctuation)
        words = [w.lower() for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)] #applying tokenization
        words = [w for w in words if w not in stop_words and w.isalnum()] # removing stopwords and only alphanumeric
        words = [w for w in words if len(w)>1] #removing if length is less than 1
        tokens = [lemma.lemmatize(w,'v')  for w in words]
        des = ' '.join(tokens)
        sent.append(des)
        name.append('unknown'+str(count))
        count+=1
    df=pd.DataFrame({'name':name,'keywords':sent})
    df.set_index('name',inplace=True)
    return df


In [37]:
text_=["""Since 2009, HIDDEN COBRA actors have leveraged their capabilities to target and compromise a range of victims; some intrusions have resulted in the exfiltration of data while others have been disruptive in nature. Commercial reporting has referred to this activity as Lazarus Group and Guardians of Peace. Tools and capabilities used by HIDDEN COBRA actors include DDoS botnets, keyloggers, remote access tools (RATs), and wiper malware. Variants of malware and tools used by HIDDEN COBRA actors include Destover, Duuzer, and Hangman."""]

In [38]:
df=pd.read_csv(r'D:\MY STUFF\rhym\data/concatenated_ta.csv',index_col='name')
df1 = text_preprocess(text_)

In [39]:
indexer = recordlinkage.Index()
indexer.full()
total = indexer.index(df, df1)
print(len(total))

93


In [40]:
#qgram and cosine distance no need of threshold
m=['jaro', 'jarowinkler', 'levenshtein', 'damerau_levenshtein', 'qgram', 'cosine', 'smith_waterman', 'lcs']
thresh=[0.40,0.50,0.55,0.60,0.65,0.70]

In [41]:
for i in m:
    for j in thresh:
        compare = recordlinkage.Compare()
        compare.string('keywords',
                       'keywords',
                       method=i,
                       threshold=j,
                       label='keywords')
        features = compare.compute(total, df,df1)
        feature_df=features[features['keywords']==1]
        if len(feature_df) >0:
            print(j,i,len(feature_df))
        else:
            print(j,i,len(feature_df))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-8ed4a6407b52>", line 9, in <module>
    features = compare.compute(total, df,df1)
  File "C:\ProgramData\Anaconda3\lib\site-packages\recordlinkage\base.py", line 865, in compute
    results = self._compute(pairs, x, x_link)
  File "C:\ProgramData\Anaconda3\lib\site-packages\recordlinkage\base.py", line 729, in _compute
    result = feat._compute(data1, data2)
  File "C:\ProgramData\Anaconda3\lib\site-packages\recordlinkage\base.py", line 449, in _compute
    result = self._compute_vectorized(*tuple(left_on + right_on))
  File "C:\ProgramData\Anaconda3\lib\site-packages\recordlinkage\compare.py", line 150, in _compute_vectorized
    c = str_sim_alg(s_left, s_right)
  File "C:\ProgramData\Anaconda3\lib\site-packages\recordlinkage\algorithms\string.py", line 30, in jar

KeyboardInterrupt: 

In [81]:
compare = recordlinkage.Compare()
compare.string('keywords',
                       'keywords',
                       method='jarowinkler',
                       threshold=0.70,
                       label='keywords')
features = compare.compute(total, df,df1)
feature_df=features[features['keywords']==1]

In [84]:
golden = features[0:20]
golden_index = feature_df.index & total

In [85]:
logreg = rl.LogisticRegressionClassifier()

# Train the classifier
logreg.fit(golden, golden_index)
print ("Intercept: ", logreg.intercept)
print ("Coefficients: ", logreg.coefficients)

Intercept:  -1.4158954705584073
Coefficients:  [1.70898777]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [86]:
result_logreg = logreg.predict(features)
len(result_logreg)
rl.confusion_matrix(total, result_logreg, len(features))

array([[17, 76],
       [ 0,  0]])

In [87]:
features['keywords'].value_counts()

0.0    76
1.0    17
Name: keywords, dtype: int64

In [49]:
features[features['keywords'] == 1]

,,keywords
name_1,name_2,
apt16,unknown0,1.0
apt17,unknown0,1.0
apt30,unknown0,1.0
blackoasis,unknown0,1.0
charming kitten,unknown0,1.0
dragonfly,unknown0,1.0
equation,unknown0,1.0
gallmaker,unknown0,1.0
lotus blossom,unknown0,1.0


In [124]:
true_linkage = pd.Series(df1['keywords'], index=df1.index)

logrg = recordlinkage.LogisticRegressionClassifier()
logrg.fit(features[true_linkage.index], true_linkage)

logrg.predict(features)

KeyError: "None of [Index(['unknown'], dtype='object', name='name')] are in the [columns]"

In [122]:
true_linkage

name
unknown    (k, e, y, w, o, r, d, s)
dtype: object

In [111]:
ecm = recordlinkage.BernoulliEMClassifier()
ecm.fit_predict(features)

AttributeError: module 'recordlinkage' has no attribute 'BernoulliEMClassifier'

In [67]:
potential_matches

,name_1,name_2,keywords


In [1]:
import recordlinkage as rl
from recordlinkage.datasets import load_krebsregister
krebs_X, krebs_true_links = load_krebsregister(missing_values=0)
krebs_X

,,cmp_firstname1,cmp_firstname2,cmp_lastname1,cmp_lastname2,cmp_sex,cmp_birthday,cmp_birthmonth,cmp_birthyear,cmp_zipcode
id1,id2,,,,,,,,,
22161,38467,1.000000,0.0,0.142857,0.0,1,0.0,1.0,0.0,0.0
38713,75352,0.000000,0.0,0.571429,0.0,1,0.0,0.0,0.0,0.0
13699,32825,0.166667,0.0,0.000000,0.0,0,1.0,1.0,1.0,0.0
22709,37682,0.285714,0.0,1.000000,0.0,1,0.0,0.0,0.0,0.0
2342,69060,0.250000,0.0,0.125000,0.0,1,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52124,53629,1.000000,0.0,0.285714,0.0,1,0.0,0.0,1.0,0.0
30007,76846,0.750000,0.0,0.000000,0.0,1,1.0,0.0,0.0,0.0
50546,59461,0.750000,0.0,0.000000,0.0,1,0.0,1.0,0.0,0.0


In [2]:
golden_pairs = krebs_X[0:5000]
golden_matches_index = golden_pairs.index & krebs_true_links

In [ ]:
logreg = rl.LogisticRegressionClassifier()
# Train the classifier
logreg.fit(golden_pairs, golden_matches_index)
print ("Intercept: ", logreg.intercept)
print ("Coefficients: ", logreg.coefficients)

In [21]:
import stringdist
stringdist.rdlevenshtein_norm('test', 'hello testing ')

0.7142857142857143

In [11]:
stringdist.osa('ab', 'bxa')

AttributeError: module 'stringdist' has no attribute 'osa'

In [ ]:
levenshtein
levenshtein_norm
rdlevenshtein
rdlevenshtein_norm